![data-x](https://raw.githubusercontent.com/afo/data-x-plaksha/master/imgsource/dx_logo.png)

---
# Cookbook 5: SQL Example

**Author list:** SINDHUJA JEYABAL, ALEXANDER FRED OJALA

**References / Sources:** 


**License Agreement:** Feel free to do whatever you want with this code

___

*This notebook is an introduction on how to work with SQL databases in Python*

In [9]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt 

#### Initialize database with filename 311_NYC.db in current working directory

In [3]:
disk_engine = create_engine('sqlite:///311_NYC.db')

#### Create DataFrame and extract data

In [ ]:
start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('311_NYC.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])

    df.index += index_start

    # Remove the un-interesting columns
    columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor',
               'CreatedDate', 'ClosedDate', 'TimeToCompletion',
               'City']

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1
    print '{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize)

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1
#     if (j == 5):
#         break

#### Clean up the data

In [ ]:
df = df.dropna()

#### Preview the Table

In [4]:
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df.head()

,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2011-06-02 08:26:00.000000,2011-06-08 22:30:00.000000,DEP,Water System,Dirty Water (WE),BRONX
1,2,2011-06-02 08:07:00.000000,2011-06-02 10:28:00.000000,DEP,Sewer,Catch Basin Search (SC2),JAMAICA
2,3,2011-06-02 08:29:00.000000,2011-06-02 11:20:00.000000,DEP,Water System,Other Water Problem (Use Comments) (WZZ),BRONX


#### Select a few columns

In [45]:
df = pd.read_sql_query('SELECT Agency, Descriptor FROM data LIMIT 3', disk_engine)
df.head()

,Agency,Descriptor
0,DEP,Dirty Water (WE)
1,DEP,Catch Basin Search (SC2)
2,DEP,Other Water Problem (Use Comments) (WZZ)


#### Filter rows using WHERE

In [51]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency = "ACS" '
                       'LIMIT 10', disk_engine)
df.head()

,ComplaintType,Descriptor,Agency
0,Damaged Tree,Child Care Information,ACS
1,Agency Issues,Family Court Issue,ACS
2,Agency Issues,Language Access Complaint,ACS
3,Agency Issues,Family Court Issue,ACS
4,Agency Issues,Family Court Issue,ACS


#### Filter multiple values in a column using IN

In [52]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency IN ("NYPD", "DOB")'
                       'LIMIT 20', disk_engine)
df.head()

,ComplaintType,Descriptor,Agency
0,General Construction/Plumbing,"Building - Vacant, Open And Unguarded",DOB
1,General Construction/Plumbing,Facade - Defective/Cracking (Ll11/98),DOB
2,Elevator,Elevator - Defective/Not Working,DOB
3,Plumbing,Failure To Retain Water/Improper Drainage- (LL...,DOB
4,General Construction/Plumbing,Sidewalk Shed/Pipe Scafford - Inadequate Defec...,DOB


#### Find Unique values using DISTINCT

In [8]:
df = pd.read_sql_query('SELECT DISTINCT Agency FROM data', disk_engine)
df.head()

,Agency
0,3-1-1
1,ACS
2,AJC
3,CAU
4,CCRB


####  Query values using COUNT(*) and GROUP_BY

In [54]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency ', disk_engine)

df.head()

,Agency,num_complaints
0,3-1-1,24090
1,ACS,114
2,AJC,9
3,CAU,10
4,CCRB,13


#### Which Department receives the most complaints?  And which receives the least?

In [6]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency '
                       'ORDER BY -num_complaints', disk_engine)
df.head()

,Agency,num_complaints
0,HPD,3459933
1,NYPD,2533007
2,DOT,1724791
3,DEP,1123418
4,DSNY,824716


#### What is the most common complaint type?

In [56]:
df = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as `num_complaints`, Agency '
                       'FROM data '
                       'GROUP BY `ComplaintType` '
                       'ORDER BY -num_complaints', disk_engine)

df.head()

,ComplaintType,num_complaints,Agency
0,Noise - Residential,975859,NYPD
1,HEATING,641651,HPD
2,Street Light Condition,570169,DOT
3,Street Condition,568285,DOT
4,HEAT/HOT WATER,493850,HPD


#### Number of complaints per city for the top 10 cities with most complaints

In [36]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                                'FROM data '
                                'GROUP BY `City` '
                       'ORDER BY -num_complaints '
                       'LIMIT 10 ', disk_engine)
df

,City,num_complaints
0,BROOKLYN,3497443
1,NEW YORK,2308782
2,BRONX,2104067
3,None,873812
4,STATEN ISLAND,566333
5,JAMAICA,160860
6,FLUSHING,126878
7,Jamaica,119738
8,ASTORIA,107936
9,Flushing,82418


#### Perform Case Insensitive Queries

In [37]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                        'FROM data '
                        'GROUP BY `City` '
                       'COLLATE NOCASE '
                       'ORDER BY -num_complaints '
                       'LIMIT 11 ', disk_engine)
df

,City,num_complaints
0,BROOKLYN,3497443
1,NEW YORK,2308782
2,BRONX,2104067
3,None,873812
4,STATEN ISLAND,566333
5,JAMAICA,280598
6,FLUSHING,209296
7,ASTORIA,177826
8,RIDGEWOOD,126352
9,CORONA,92568
